In [1]:
import plom
import numpy as np

## Data

In [2]:
np.random.seed(24)
n_x = 5
N_model = 200
N_test = 20

X = np.random.rand(n_x, N_model + N_test)
y = (X[0] * 40 + 100) + (X[1] + 5) + (X[2] * 0.01)

data = np.vstack((X, y)).T

data_model = data[:N_model]
data_test = data[-N_test:]

cond_cols = range(n_x)
qoi_col = n_x

## Bandwidth joint optimization

In [3]:
np.random.seed()
bw_opt, mse_opt = plom.conditioning_jointly_optimal_bw(data_model, cond_cols, qoi_col, ga_workers=-1, return_mse=True)
bw_silverman, mse_silverman = plom.conditioning_silverman_bw(data_model, cond_cols, qoi_col, return_mse=True)

c:\ProgramData\anaconda3\Lib\site-packages\scipy\optimize\_differentialevolution.py:387: UserWarning: differential_evolution: the 'workers' keyword has overridden updating='immediate' to updating='deferred'
  with DifferentialEvolutionSolver(func, bounds, args=args,


In [4]:
print(f"Optimal bandwidth: {bw_opt}")
print(f"Optimal MSE: {mse_opt}\n")
print(f"Silverman bandwidth: {bw_silverman}\n")
print(f"Silverman MSE: {mse_silverman}\n")

Optimal bandwidth: [1.00000008e-09 5.47036501e+02 1.67344107e+02 4.06340169e+02
 2.46336581e+02]
Optimal MSE: 0.20612237419642096

Silverman bandwidth: 0.5492802716530588

Silverman MSE: 9.938308983436842



In [5]:
x_test = data_test[:, cond_cols]
y_true = data_test[:, qoi_col]
y_pred_silverman = plom.forward_model(x_test, data_model, qoi_col, h=bw_silverman)
error_silverman = np.round(np.abs(y_pred_silverman-y_true)/np.abs(y_true)*100, 3)
y_pred_opt = plom.forward_model(x_test, data_model, qoi_col, h=bw_opt)
error_opt = np.round(np.abs(y_pred_opt-y_true)/np.abs(y_true)*100, 3)
mse_pred_opt = plom.mse(y_pred_opt, y_true)

In [6]:
for test_case in range(N_test):
    print(f"y_true             = {y_true[test_case]}")
    print(f"y_pred (optimal)   = {y_pred_opt[test_case]} ({error_opt[test_case]}%)")
    print(f"y_pred (silverman) = {y_pred_silverman[test_case]} ({error_silverman[test_case]}%)\n")

print(f"\nPrediction MSE (optimal) = {mse_pred_opt}")

y_true             = 116.88816310659685
y_pred (optimal)   = 117.40241662659318 (0.44%)
y_pred (silverman) = 119.57471914158145 (2.298%)

y_true             = 140.1963915146468
y_pred (optimal)   = 140.41875618833356 (0.159%)
y_pred (silverman) = 138.72481070176636 (1.05%)

y_true             = 106.95109360255577
y_pred (optimal)   = 106.68540846668652 (0.248%)
y_pred (silverman) = 109.77812885225666 (2.643%)

y_true             = 105.8677292092009
y_pred (optimal)   = 106.04503008689751 (0.167%)
y_pred (silverman) = 109.07210903599247 (3.027%)

y_true             = 117.92503674547793
y_pred (optimal)   = 118.2114483543469 (0.243%)
y_pred (silverman) = 120.35187505281928 (2.058%)

y_true             = 120.45283990376232
y_pred (optimal)   = 120.79059736814203 (0.28%)
y_pred (silverman) = 120.49509169634993 (0.035%)

y_true             = 145.72329116249972
y_pred (optimal)   = 145.66151567845054 (0.042%)
y_pred (silverman) = 141.1092440237039 (3.166%)

y_true             = 137.723870435

## Bandwidth marginal optimization

In [7]:
np.random.seed()
bw_opt_marg, mse_opt_marg = plom.conditioning_marginally_optimal_bw(
    data_model, cond_cols, qoi_col, ranking_kfolds=10, opt_kfolds=1, 
    opt_cycles=10, ga_bounds=(1e-06, 1e6), shuffle=False, split_seed=None, 
    logscale=True, ga_workers=1, verbose=True)
bw_silverman, mse_silverman = plom.conditioning_silverman_bw(data_model, cond_cols, qoi_col)

OPTMIZING SINGLE BANDWIDTHS FOR RANKING PURPOSES

******************

Ranking fold 1/10

Optimizing bandwidth for variable 0
H_0 = [0.04001626]
MSE (from optimizer) = 0.137 

Optimizing bandwidth for variable 1
H_1 = [326930.5897221]
MSE (from optimizer) = 171.68 

Optimizing bandwidth for variable 2
H_2 = [0.86264207]
MSE (from optimizer) = 166.983 

Optimizing bandwidth for variable 3
H_3 = [887993.44454949]
MSE (from optimizer) = 171.68 

Optimizing bandwidth for variable 4
H_4 = [688771.83349993]
MSE (from optimizer) = 171.68 

******************

Ranking fold 2/10

Optimizing bandwidth for variable 0
H_0 = [0.0632734]
MSE (from optimizer) = 0.112 

Optimizing bandwidth for variable 1
H_1 = [489042.24360447]
MSE (from optimizer) = 181.8 

Optimizing bandwidth for variable 2
H_2 = [2.77400179]
MSE (from optimizer) = 181.718 

Optimizing bandwidth for variable 3
H_3 = [0.00392485]
MSE (from optimizer) = 140.602 

Optimizing bandwidth for variable 4
H_4 = [0.07729742]
MSE (from optimi

In [8]:
print(f"Optimal bandwidth: {bw_opt_marg}")
print(f"Optimal MSE: {mse_opt_marg}\n")
print(f"Silverman bandwidth: {bw_silverman}")
print(f"Silverman MSE: {mse_silverman}\n")

Optimal bandwidth: [3.82379596e-02 4.55518124e-01 7.33324412e+05 1.24428769e+00
 1.35968389e+00]
Optimal MSE: 0.04673238316816038

Silverman bandwidth: 0.5492802716530588
Silverman MSE: 0.5492802716530588



In [9]:
x_test = data_test[:, cond_cols]
y_true = data_test[:, qoi_col]
y_pred_silverman = plom.forward_model(x_test, data_model, qoi_col, h=bw_silverman)
error_silverman = np.round(np.abs(y_pred_silverman-y_true)/np.abs(y_true)*100, 3)
y_pred_opt_marg = plom.forward_model(x_test, data_model, qoi_col, h=bw_opt_marg)
error_opt_marg = np.round(np.abs(y_pred_opt_marg-y_true)/np.abs(y_true)*100, 3)
mse_pred_opt_marg = plom.mse(y_pred_opt_marg, y_true)

In [10]:
for test_case in range(N_test):
    print(f"y_true             = {y_true[test_case]}")
    print(f"y_pred (optimal)   = {y_pred_opt_marg[test_case]} ({error_opt_marg[test_case]})%")
    print(f"y_pred (silverman) = {y_pred_silverman[test_case]} ({error_silverman[test_case]})%\n")
print(f"\nPrediction MSE (optimal) = {mse_pred_opt_marg}")

y_true             = 116.88816310659685
y_pred (optimal)   = 116.81747842103239 (0.06)%
y_pred (silverman) = 119.57471914158145 (2.298)%

y_true             = 140.1963915146468
y_pred (optimal)   = 140.1616951160389 (0.025)%
y_pred (silverman) = 138.72481070176636 (1.05)%

y_true             = 106.95109360255577
y_pred (optimal)   = 106.98231662698007 (0.029)%
y_pred (silverman) = 109.77812885225666 (2.643)%

y_true             = 105.8677292092009
y_pred (optimal)   = 105.96515197020892 (0.092)%
y_pred (silverman) = 109.07210903599247 (3.027)%

y_true             = 117.92503674547793
y_pred (optimal)   = 117.75786655227886 (0.142)%
y_pred (silverman) = 120.35187505281928 (2.058)%

y_true             = 120.45283990376232
y_pred (optimal)   = 120.76611267669382 (0.26)%
y_pred (silverman) = 120.49509169634993 (0.035)%

y_true             = 145.72329116249972
y_pred (optimal)   = 145.6091785309378 (0.078)%
y_pred (silverman) = 141.1092440237039 (3.166)%

y_true             = 137.7238704357